### 뉴스 데이터 전처리
#### predict용 data 만들기

In [1]:
# 뉴스 raw csv 파일 합치기
import os
import pandas as pd

folder_path = "../data/raw/"

# (NEWS)가 포함된 Excel 파일만 선택
excel_files = [f for f in os.listdir(folder_path) if f.endswith(".xlsx") and "(NEWS)" in f]

df_list = []
for file in excel_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_excel(file_path) 
    df_list.append(df)

# 파일이 있을 때만 합치기
if df_list:
    merged_df = pd.concat(df_list, ignore_index=True)

    # 합친 결과를 CSV로 저장
    output_file = os.path.join(folder_path, "../processed/news_raw.csv")
    merged_df.to_csv(output_file, index=False, encoding="utf-8-sig")

    print(f"[완료] (NEWS) 포함된 {len(excel_files)}개 엑셀 파일을 합쳐서 news_raw.csv 저장했습니다.")
else:
    print("[알림] (NEWS) 가 포함된 엑셀 파일이 없습니다.")

[완료] (NEWS) 포함된 4개 엑셀 파일을 합쳐서 news_raw.csv 저장했습니다.


In [3]:
# 전처리
import pandas as pd
df = pd.read_csv("../data/processed/news_raw.csv", encoding="utf-8-sig")

# content가 비어있거나 NaN인 행 제거
df_clean = df.dropna(subset=["content"])          # NaN 값 제거
df_clean = df_clean[df_clean["content"].str.strip() != ""]  # 공백만 있는 경우 제거
print(f"[Step1] content 비어있는 행 제거: {len(df)} → {len(df_clean)}")

# url + content 기준 중복 제거
df_clean = df_clean.drop_duplicates(subset=["url", "content"], keep="first")
print(f"[Step2] url+content 중복 제거 후 행 수: {len(df_clean)}")

# 키워드 포함된 기사 제거 (예: 오늘의 뉴스, 헤드라인 등)
keywords = ["오늘의 국회일정", "오늘의 주요일정", "미리보는 이데일리 신문",
            "연합뉴스 이 시각 헤드라인", "데일리안 1분뉴스", "헤드라인", "온라인 핫 뉴스만 콕콕"]

# regex=False 옵션으로 특수문자 처리
pattern = "|".join(keywords)
df_clean = df_clean[~(df_clean["content"].str.contains(pattern, na=False) |
                      df_clean["title"].str.contains(pattern, na=False))]
print(f"[Step3] 키워드 포함 기사 제거 후 행 수: {len(df_clean)}")

df_clean.to_csv("../data/processed/news_pre.csv", index=False, encoding="utf-8-sig")
print("[완료] news_pre.csv 저장 완료")

[Step1] content 비어있는 행 제거: 28270 → 28256
[Step2] url+content 중복 제거 후 행 수: 28023
[Step3] 키워드 포함 기사 제거 후 행 수: 27969
[완료] news_pre.csv 저장 완료


#### train/test용 data 만들기

#### 뉴스 기사 분류 모델의 학습 데이터 구축 및 점진적 개선을 위한 파이프라인
##### 초기 소규모 데이터셋에서 시작해 딥러닝 모델의 예측을 활용한 반자동 레이블링과 점진적 학습을 통해 대규모 학습 데이터셋을 구축하는 과정을 설명함

단계 1 : 초기 학습 데이터셋 구축 및 기반 모델 학습

1. 초기 학습 데이터셋 준비
- 원본 데이터(news_pre.csv)에서 각 카테고리별로 100개의 기사 무작위 샘플링(400_ver1.csv)
- 총 400개의 샘플(400_ver1.csv)에 대해 'category'와 'MAIN_NAME'을 수동으로 레이블링함
- 이 데이터는 모델학습의 기반이 되는 시드 데이터셋 역할을 함

2. 첫번째 분류모델 학습 및 최적 모델 저장
- 수동으로 레이블링된 400개의 데이터를 이용해 분류모델을 학습시킴
- 학습과정에서 검증 데이터셋을 이용해 모델의 성능을 평가하고, 가장 성능이 우수한 best model을 저장함

단계 2 : 모델을 활용한 데이터 확장 및 검수 

1. 2차 샘플링 및 예측 기반 레이블링
- 원본 데이터(news_pre.csv)에서 동일한 방식으로 새로운 샘플 400개 추출함(400_ver2.csv)
- 저장된 첫번째 best model을 사용해서 400_ver2.csv의 'category'와 'MAIN_NAME'을 예측함
- 예측된 레이블을 사람이 직접 검수해서 오류를 수정함

2. 데이터셋 확장 및 모델 성능 개선
- 검수가 완료된 400_ver2.csv와 기존의 400_ver1.csv을 병합하여 총 800개의 데이터셋 구축(800 w 정답라벨.csv)
- 확장된 800개의 데이터셋으로 분류모델을 다시 학습시키고, 성능이 가장 우수한 best model을 새로 저장함

단계 3 : 반복적인 모델 고도화

1. 추가 샘플링 및 검수
- 원본 데이터(news_pre.csv)에서 동일한 방식으로 새로운 샘플 400개 추출한 후, 2단계에서 개선된 모델로 레이블을 예측함
- 예측결과를 수동으로 검수 및 수정함
2. 점진적 학습 및 모델 고도화
- 800개의 데이터와 새로 검수한 400개의 데이터를 합쳐 총 1200개의 데이터셋 구축(1200 w 정답라벨.csv)
- 이 과정을 반복하여 최종적으로 1600개의 정답 레이블이 확보된 데이터셋 구축(1600 w 정답라벨.csv)
3. 최종 모델 학습
- 이 단계에서 확보된 1600개의 고품질 데이터셋으로 분류모델을 다시 학습시킨 후, 최종 예측에 사용할 최적의 best model 저장

단계 4 : 대규모 최종 학습 데이터셋 생성

- 대규모 학습을 위해 원본데이터에서 카테고리별 1250개씩, 총 5000개의 기사를 무작위로 샘플링(5000_w_only_content.csv)
- 3단계에서 학습된 best model을 사용해서 5000개의 샘플의 레이블을 예측함
- 예측된 결과를 최종적으로 검수하여 5000_w_label.csv로 저장
- 5000_w_label.csv 파일은 최종 모델 학습 및 평가를 위한 대규모의 최종 데이터셋으로 활용됨

In [ ]:
# 400건(category별 랜덤 100건 추출) 샘플링 후 라벨링 수작업
import pandas as pd

INPUT_FILE_PATH = '../data/processed/news_pre.csv'
OUTPUT_FILE_PATH = '../data/processed/400_ver1.csv'

CATEGORY_COLUMN = 'category'  # 그룹화할 기준 컬럼
N_SAMPLES = 100             # 각 카테고리별로 추출할 기사 수

try:
    df = pd.read_csv(INPUT_FILE_PATH)
    print(f"✅ '{INPUT_FILE_PATH}' 파일을 성공적으로 불러왔습니다.")
    print(f"원본 데이터 크기: {len(df)} 행")
except FileNotFoundError:
    print(f"오류: '{INPUT_FILE_PATH}' 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
    exit()

#  카테고리별 랜덤 샘플링
sampled_df = df.groupby(CATEGORY_COLUMN, group_keys=False).apply(
    lambda x: x.sample(n=min(len(x), N_SAMPLES), random_state=42)
)
sampled_df.reset_index(drop=True, inplace=True)

# 각 카테고리별로 몇 건씩 추출되었는지 확인
print("\n--- 각 카테고리별 추출된 기사 수 ---")
print(sampled_df[CATEGORY_COLUMN].value_counts())
print("------------------------------------")
sampled_df = sampled_df[['content']]
sampled_df.to_csv(OUTPUT_FILE_PATH, index=False, encoding='utf-8-sig')
print(f"\n샘플링된 데이터가 '{OUTPUT_FILE_PATH}' 파일에 성공적으로 저장되었습니다.")

✅ '../../data/processed/news_pre.csv' 파일을 성공적으로 불러왔습니다.
원본 데이터 크기: 27969 행

--- 각 카테고리별 추출된 기사 수 ---
category
개인정보보호법,정보통신망법                          100
아동복지법                                   100
자본시장법,특정금융정보법,전자금융거래법,전자증권법,금융소비자보호법    100
중대재해처벌법                                 100
Name: count, dtype: int64
------------------------------------

샘플링된 데이터가 '../../data/processed/400_ver1.csv' 파일에 성공적으로 저장되었습니다.


C:\Users\user\AppData\Local\Temp\ipykernel_16240\2669191907.py:19: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = df.groupby(CATEGORY_COLUMN, group_keys=False).apply(


In [ ]:
# 5000건(category별 랜덤 1250건 추출) 샘플링
import pandas as pd

INPUT_FILE_PATH = '../data/processed/news_pre.csv'
OUTPUT_FILE_PATH = '../data/processed/5000_w_only_content.csv'

CATEGORY_COLUMN = 'category'  # 그룹화할 기준 컬럼
N_SAMPLES = 1250              # 각 카테고리별로 추출할 기사 수

try:
    df = pd.read_csv(INPUT_FILE_PATH)
    print(f"✅ '{INPUT_FILE_PATH}' 파일을 성공적으로 불러왔습니다.")
    print(f"원본 데이터 크기: {len(df)} 행")
except FileNotFoundError:
    print(f"오류: '{INPUT_FILE_PATH}' 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
    exit()

# 카테고리별 랜덤 샘플링
sampled_df = df.groupby(CATEGORY_COLUMN, group_keys=False).apply(
    lambda x: x.sample(n=min(len(x), N_SAMPLES), random_state=42)
)
sampled_df.reset_index(drop=True, inplace=True)
sampled_df = sampled_df[['content']]
sampled_df.to_csv(OUTPUT_FILE_PATH, index=False, encoding='utf-8-sig')
print(f"\n샘플링된 데이터가 '{OUTPUT_FILE_PATH}' 파일에 성공적으로 저장되었습니다.")

✅ '../../data/processed/news_pre.csv' 파일을 성공적으로 불러왔습니다.
원본 데이터 크기: 27969 행

샘플링된 데이터가 '../../data/processed/5000_w_only_content.csv' 파일에 성공적으로 저장되었습니다.


C:\Users\user\AppData\Local\Temp\ipykernel_16240\464099888.py:19: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = df.groupby(CATEGORY_COLUMN, group_keys=False).apply(
